In [4]:
from reasoner.cypher import get_query

## Predicates

This library converts TRAPI into cypher.  In this process, it takes into account biolink model's predicates:

In [5]:
qg={
  "nodes": {
    "n0": {
      "category": [
        "biolink:ChemicalSubstance"
      ],
      "name": "Chemical Substance"
    },
    "n1": {
      "name": "RHOBTB2",
      "id": "NCBIGene:23221"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicate": [
        "biolink:affects_expression_of"
      ]
    }
  }
}

In [6]:
get_query(qg)

'MATCH (`n0`:`biolink:ChemicalSubstance`)-[`e0`:`biolink:affects_expression_of`|`biolink:increases_expression_of`|`biolink:decreases_expression_of`]->(`n1` {`id`: "NCBIGene:23221"}) WITH {node_bindings: {n0: (CASE WHEN n0 IS NOT NULL THEN [{id: n0.id}] ELSE [] END), n1: (CASE WHEN n1 IS NOT NULL THEN [{id: n1.id}] ELSE [] END)}, edge_bindings: {e0: [ei IN collect(DISTINCT `e0`.id) WHERE ei IS NOT null | {id: ei}]}} AS result, {nodes: collect(DISTINCT `n0`) + collect(DISTINCT `n1`), edges: collect(DISTINCT `e0`)} AS knowledge_graph UNWIND knowledge_graph.nodes AS knode UNWIND knowledge_graph.edges AS kedge WITH collect(DISTINCT result) AS results, {nodes: apoc.map.fromLists([n IN collect(DISTINCT knode) | n.id], [n IN collect(DISTINCT knode) | {category: labels(n), name: n.name, attributes: [key in apoc.coll.subtract(keys(n), ["id", "category"]) | {name: key, type: COALESCE({publications: "EDAM:data_0971"}[key], "NA"), value: n[key]}]}]), edges: apoc.map.fromLists([e IN collect(DISTINCT

## Categories

Note that it doesn't do anything with categories.  Here we are making n0 a Named Thing as opposed to a Chemical Substance, and the cypher query doesn't do any subsetting.  It assumes that a ChemicalSubstance node in the underlying graph will also have a NamedThing label.

In [7]:
qg={
  "nodes": {
    "n0": {
      "category": [
        "biolink:NamedThing"
      ],
    },
    "n1": {
      "name": "RHOBTB2",
      "id": "NCBIGene:23221"
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1",
      "predicate": [
        "biolink:affects_expression_of"
      ]
    }
  }
}

In [8]:
get_query(qg)

'MATCH (`n0`:`biolink:NamedThing`)-[`e0`:`biolink:affects_expression_of`|`biolink:increases_expression_of`|`biolink:decreases_expression_of`]->(`n1` {`id`: "NCBIGene:23221"}) WITH {node_bindings: {n0: (CASE WHEN n0 IS NOT NULL THEN [{id: n0.id}] ELSE [] END), n1: (CASE WHEN n1 IS NOT NULL THEN [{id: n1.id}] ELSE [] END)}, edge_bindings: {e0: [ei IN collect(DISTINCT `e0`.id) WHERE ei IS NOT null | {id: ei}]}} AS result, {nodes: collect(DISTINCT `n0`) + collect(DISTINCT `n1`), edges: collect(DISTINCT `e0`)} AS knowledge_graph UNWIND knowledge_graph.nodes AS knode UNWIND knowledge_graph.edges AS kedge WITH collect(DISTINCT result) AS results, {nodes: apoc.map.fromLists([n IN collect(DISTINCT knode) | n.id], [n IN collect(DISTINCT knode) | {category: labels(n), name: n.name, attributes: [key in apoc.coll.subtract(keys(n), ["id", "category"]) | {name: key, type: COALESCE({publications: "EDAM:data_0971"}[key], "NA"), value: n[key]}]}]), edges: apoc.map.fromLists([e IN collect(DISTINCT kedge)